In [ ]:
import torch
import torch.nn as nn
import numpy as np
import time

from torch.autograd import Variable
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader

import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets

from torch.utils.data import Dataset

from sklearn.metrics import accuracy_score,roc_auc_score
import os
import nibabel as nib
import argparse
from utils import AverageMeter
from distutils.version import LooseVersion
import math
from vgg import vgg16_bn
from resnet import resnet18
from vgg_dcf import vgg16_bn_dcf
from resnet_dcf import resnet18_dcf
import matplotlib.pyplot as plt

# define hyper-parameters
# Data related arguments
NUM_CLASSES = 10
NUM_WORKERS = 4

# optimization related arguments
BATCH_SIZE = 100
NUM_EPOCHES = 100
LR = 1e-4
    
def train(train_loader, model, criterion, optimizer, epoch):
    losses = AverageMeter()

    model.train()
    iteration = 0
    for images,labels in train_loader:

        image = Variable(images.float()).cuda()
        label = Variable(labels).long().cuda()

        out = model(image)
        loss = criterion(out, label)
        losses.update(loss.data[0],image.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if iteration%100==0:
            print('   * i {} | Training Loss: {losses.avg:.3f}'.format(iteration, losses=losses))
        iteration +=1
    print('   * EPOCH {epoch} | Training Loss: {losses.avg:.3f}'.format(epoch=epoch, losses=losses))
    return losses.avg

def validation(val_loader, model, epoch):
    model.eval()
    correct = 0
    total = 0
    for images, labels in val_loader:
        image = Variable(images).float().cuda()
        label = Variable(labels).long().data.cpu().numpy()
        out = model(image)
        with torch.no_grad():
            _, predicted = torch.max(out.data, 1)
            total += labels.size(0)
            correct += (predicted.data.cpu().numpy() == labels).sum()

    print('   * EPOCH {epoch} | Validation accuracy: {losses} '.format(epoch=epoch, losses=float(correct) / float(total)))
    return  float(correct) / float(total)      # print(out[0,0])
            # break

def save_checkpoint(state, epoch, path):
    filename = path + '/' + str(epoch) + '_checkpoint.pth.tar'
    print(filename)
    torch.save(state, filename)

def main():

    # model = vgg16_bn(pretrained = False, num_classes = NUM_CLASSES)
    # model = vgg16_bn_dcf(num_classes = NUM_CLASSES)
    # model = resnet18(num_classes = NUM_CLASSES)
    # model = resnet18_dcf(num_classes = NUM_CLASSES)
    model = resnet18_dcf(num_classes = NUM_CLASSES)
    
    CKPT = './saved_models/resnet18_dcf/'
    if not os.path.isdir(CKPT):
        os.makedirs(CKPT)

    model.cuda()
    cudnn.benchmark = True

    # collect the number of parameters in the network
    print("------------------------------------------")
    num_para = 0
    for name, param in model.named_parameters():
        num_mul = 1
        for x in param.size():
            num_mul *= x
        num_para += num_mul

    print(model)
    print("Number of trainable parameters %d in Model" % (num_para))
    print("------------------------------------------")

    # set the optimizer and loss
    optimizer = optim.Adam(model.parameters(), lr=LR)

    # set loss function
    criterion = nn.CrossEntropyLoss()

    # loading data
    data_transforms_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        # transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
    ])
    dataset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = data_transforms_train)
    train_loader = DataLoader(dataset,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            num_workers=NUM_WORKERS)
    
    data_transforms_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
    ])
    dataset_val = torchvision.datasets.CIFAR10(root = './data', train = False, download =True,  transform = data_transforms_test)
    validation_loader = DataLoader(dataset_val,
                            batch_size=BATCH_SIZE,
                            shuffle=False,
                            num_workers=NUM_WORKERS)

    print("Start training ...")
    train_loss = []
    avg_accu = []

    best_epoch=0
    best_accu= 0
    for epoch in range(1, NUM_EPOCHES + 1):
        start_time = time.time()
        loss = train(train_loader, model, criterion, optimizer, epoch)
        train_loss.append(loss)
        
        if epoch % 10 == 0:
            save_checkpoint({'epoch': epoch, 'state_dict': model.state_dict(), 'opt_dict': optimizer.state_dict()}, epoch, CKPT)
            filename = CKPT + '/' + str(epoch) + '_checkpoint.pth.tar'

            checkpoint = torch.load(filename)
            state_dict = checkpoint['state_dict']
            model.load_state_dict(state_dict)
        accu = validation(validation_loader, model, epoch)
        avg_accu.append(accu)
        
        if accu > best_accu:
            best_epoch = epoch
            best_accu = accu
        elapsed_time = time.time() - start_time
        print('epoch time ' +str(time.strftime("%H:%M:%S", time.gmtime(elapsed_time))) + ', remaining time ' +str(time.strftime("%H:%M:%S", time.gmtime(elapsed_time*(NUM_EPOCHES - epoch)))))
        if epoch % 5 ==0 or epoch ==NUM_EPOCHES:
            x= range(1, epoch+1)

            plt.figure(1)

            plt.plot(x, train_loss, label='loss')
            plt.plot(x, avg_accu, label = 'val_accu')
            plt.legend ()
            plt.xlabel('epoch')

            imgname = CKPT + '/figure.png'
            plt.savefig(imgname)
            plt.gcf().clear()
    print("Training Done")
    print("Best epoch " +str(best_epoch) + ', auc score '+str(best_accu))

if __name__ == '__main__':
    assert LooseVersion(torch.__version__) >= LooseVersion('0.3.0'), \
        'PyTorch>=0.3.0 is required'

    main()


------------------------------------------
ResNet(
  (conv1): Conv_DCF()
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv_DCF()
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv_DCF()
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv_DCF()
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv_DCF()
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv_DCF()
      (bn1): BatchNorm2d(128, eps=1e-05, 

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:53: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


   * i 0 | Training Loss: 2.325
   * i 100 | Training Loss: 2.101
   * i 200 | Training Loss: 1.965
   * i 300 | Training Loss: 1.882
   * i 400 | Training Loss: 1.826
   * EPOCH 1 | Training Loss: 1.783
   * EPOCH 1 | Validation accuracy: 0.3831 
epoch time 00:00:15, remaining time 00:25:20
   * i 0 | Training Loss: 1.685
   * i 100 | Training Loss: 1.554
   * i 200 | Training Loss: 1.532
   * i 300 | Training Loss: 1.517
   * i 400 | Training Loss: 1.501
   * EPOCH 2 | Training Loss: 1.488
   * EPOCH 2 | Validation accuracy: 0.4413 
epoch time 00:00:14, remaining time 00:23:10
   * i 0 | Training Loss: 1.331
   * i 100 | Training Loss: 1.411
   * i 200 | Training Loss: 1.398
   * i 300 | Training Loss: 1.389
   * i 400 | Training Loss: 1.377
   * EPOCH 3 | Training Loss: 1.371
   * EPOCH 3 | Validation accuracy: 0.4516 
epoch time 00:00:14, remaining time 00:23:06
   * i 0 | Training Loss: 1.409
   * i 100 | Training Loss: 1.301
   * i 200 | Training Loss: 1.300
   * i 300 | Training

   * EPOCH 28 | Training Loss: 0.575
   * EPOCH 28 | Validation accuracy: 0.7599 
epoch time 00:00:14, remaining time 00:17:29
   * i 0 | Training Loss: 0.581
   * i 100 | Training Loss: 0.550
   * i 200 | Training Loss: 0.553
   * i 300 | Training Loss: 0.560
   * i 400 | Training Loss: 0.567
   * EPOCH 29 | Training Loss: 0.573
   * EPOCH 29 | Validation accuracy: 0.7556 
epoch time 00:00:14, remaining time 00:17:15
   * i 0 | Training Loss: 0.570
   * i 100 | Training Loss: 0.530
   * i 200 | Training Loss: 0.548
   * i 300 | Training Loss: 0.551
   * i 400 | Training Loss: 0.554
   * EPOCH 30 | Training Loss: 0.556
./saved_models/resnet18_dcf//30_checkpoint.pth.tar
   * EPOCH 30 | Validation accuracy: 0.7525 
epoch time 00:00:14, remaining time 00:17:02
   * i 0 | Training Loss: 0.508
   * i 100 | Training Loss: 0.543
   * i 200 | Training Loss: 0.547
   * i 300 | Training Loss: 0.547
   * i 400 | Training Loss: 0.547
   * EPOCH 31 | Training Loss: 0.545
   * EPOCH 31 | Validation 

   * i 0 | Training Loss: 0.326
   * i 100 | Training Loss: 0.362
   * i 200 | Training Loss: 0.360
   * i 300 | Training Loss: 0.360
   * i 400 | Training Loss: 0.362
   * EPOCH 56 | Training Loss: 0.367
   * EPOCH 56 | Validation accuracy: 0.8017 
epoch time 00:00:14, remaining time 00:10:37
   * i 0 | Training Loss: 0.302
   * i 100 | Training Loss: 0.343
   * i 200 | Training Loss: 0.350
   * i 300 | Training Loss: 0.356
   * i 400 | Training Loss: 0.363
   * EPOCH 57 | Training Loss: 0.365
   * EPOCH 57 | Validation accuracy: 0.7957 
epoch time 00:00:14, remaining time 00:10:27
   * i 0 | Training Loss: 0.424
   * i 100 | Training Loss: 0.347
   * i 200 | Training Loss: 0.348
   * i 300 | Training Loss: 0.354
   * i 400 | Training Loss: 0.359
   * EPOCH 58 | Training Loss: 0.358
   * EPOCH 58 | Validation accuracy: 0.7917 
epoch time 00:00:14, remaining time 00:10:11
   * i 0 | Training Loss: 0.448
   * i 100 | Training Loss: 0.348
   * i 200 | Training Loss: 0.346
   * i 300 | Tr

   * i 300 | Training Loss: 0.251
   * i 400 | Training Loss: 0.253
   * EPOCH 83 | Training Loss: 0.255
   * EPOCH 83 | Validation accuracy: 0.807 
epoch time 00:00:14, remaining time 00:04:07
   * i 0 | Training Loss: 0.188
   * i 100 | Training Loss: 0.244
   * i 200 | Training Loss: 0.242
   * i 300 | Training Loss: 0.243
   * i 400 | Training Loss: 0.244
   * EPOCH 84 | Training Loss: 0.247
   * EPOCH 84 | Validation accuracy: 0.7992 
epoch time 00:00:14, remaining time 00:03:53
   * i 0 | Training Loss: 0.328
   * i 100 | Training Loss: 0.244
   * i 200 | Training Loss: 0.246
   * i 300 | Training Loss: 0.247
   * i 400 | Training Loss: 0.248
   * EPOCH 85 | Training Loss: 0.249
   * EPOCH 85 | Validation accuracy: 0.8111 
epoch time 00:00:14, remaining time 00:03:38
   * i 0 | Training Loss: 0.186
   * i 100 | Training Loss: 0.241
   * i 200 | Training Loss: 0.236
   * i 300 | Training Loss: 0.239
   * i 400 | Training Loss: 0.244
   * EPOCH 86 | Training Loss: 0.250
   * EPOCH 